# Session 1: MPS class

Some requirements for this library component. We use the `numpy` library and import algorithms to operate on matrix product states, such as scalar produts, expectation values, norms, etc.

In [ ]:
# file: mps/state.py
import numpy as np
from mps import expectation

## General idea

A Matrix-Product State (abbreviated MPS), is a representation of quantum states as a one-dimensional contraction of tensors. Imagine the following quantum state

<img src="figures/wavefunction.svg" style="max-width: 50%; width:400px">

The state has four physical indices, so that we can write it as
$$|\Psi\rangle = \sum_{i_1,i_2,i_3,i_4} \Psi_{i_1,i_2,i_3,i_4} |i_1,i_2,i_3,i_4\rangle.$$
If the physical dimension of each component is $d$, that is $i_k \in \{0,1,\ldots, d-1\}$, this means that the total state requires $d^4$ complex numbers to be represented.


The MPS representation uses a small number of tensors (matrices) to reconstruct the previous state as

<img src="figures/four-site-mps.svg" style="max-width: 50%; width: 400px">

Now the state reads
$$|\Psi\rangle = \sum_{i's, \alpha's} A^{i_1}_{\alpha_1} B^{i_2}_{\alpha_1,\alpha_2} C^{i_4}_{\alpha_2,\alpha_3} D^{i_3}_{\alpha_3}|i_1,i_2,i_3,i_4\rangle.$$
This can be simply written in a more compact fashion as
$$
|\Psi\rangle = A^{i_1} B^{i_2} C^{i_3} D^{i_4}|i_1,i_2,i_3,i_4\rangle,
$$
assuming that the $\{A,B,C,D\}$ are either vectors or matrices, labeled by the physical index $i_k$, and that we contract over repeated indices.

If we are allowed matrices of any size, the MPS decomposition is exact. This can be seen from a recursive use of the Schmidt decomposition. The Schmidt basis is a decomposition of a bipartite quantum state $|\Psi_{AB}\rangle$ as a superposition of two different sets of states for the "A" and "B" components
$$|\Psi_{AB}\rangle = \sum_{r=1}^R \sqrt{\lambda_r}|\phi^r_A\rangle|\phi^r_B\rangle,$$
with the guarantees that the number of components $R$ is minimal and the many-body states for A and B are orthonormal basis
$$\langle\phi_A^r|\phi_A^s\rangle = \delta_{rs},\;\langle\phi_B^r|\phi_B^s\rangle=\delta_{rs}.$$

Following a variant of the reasoning by [G. Vidal (PRL 2003)](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.91.147902), we can apply a decomposition of this sort to our four-component state above, obtaining something of the sort
$$|\Psi\rangle = \sum_{\alpha_1\alpha_2\alpha_3} \sqrt{\lambda_{\alpha_1}\lambda_{\alpha_2}\lambda_{\alpha_3}\lambda_{\alpha_4}}|\phi_A^{\alpha_1}\rangle|\phi_B^{\alpha_1,\alpha_2}\rangle|\phi_C^{\alpha_2,\alpha_3}\rangle|\phi_D^{\alpha_3}\rangle,$$
with different Schmidt numbers for each of the bipartitions $(1,234), (12,34), (123,4)$ and basis that are labeled according to the bipartitions they appear at. Projecting the states above, we find a possible choice of our matrix-product-states tensors
$$A^{i_1}_{1,\alpha_1} = \langle i_1|\phi^{\alpha_1}_A\rangle$$
$$B^{i_1}_{\alpha_1,\alpha_2} = \sqrt{\lambda_{\alpha_1}}\langle i_2|\phi^{\alpha_1,\alpha_2}_B\rangle$$
$$C^{i_1}_{\alpha_2,\alpha_3} = \sqrt{\lambda_{\alpha_2}}\langle i_3|\phi^{\alpha_2,\alpha_3}_C\rangle$$
$$D^{i_1}_{\alpha_3,1} = \sqrt{\lambda_{\alpha_3}}\langle i_4|\phi^{\alpha_3}_D\rangle,$$
although other choices are feasible.

## Wavefunction to MPS

Let us begin our first MPS related code by creating this decomposition in a more systematic way. Our starting point is a generic state $|\Psi\rangle = \sum_{a,b}\Psi_{a,b}|a,b\rangle$ which we want to make into a Schmidt basis. One way would be to construct the reduced density matrix for the "a" part. This matrix contains the Schmidt numbers and the Schmidt basis for that part
$$\rho_A = \sum_b \Psi_{a,b}\Psi_{a',b}^* |a\rangle\langle a'|=\sum_r \lambda_r |\phi_A^r\rangle\langle\phi_A^r|.$$
We can then take the $|\phi_A^r\rangle$ basis and deduce the corresponding states for the $B$ component.

A simpler and more efficient way is to compute the singular value decomposition of the "matrix" $\Psi_{a,b}$
$$\Psi_{a,b} = U_{a,c} S_{c,d} V_{d,b},$$
where $U$ and $V$ are two unitary matrices and $S$ is a diagonal matrix $S_{c,d} = s_c \delta_{c,d}$ of non-negative numbers. Once we sort out which values $s_c$ are non-zero, we can construct a possible Schmidt basis and numbers as follows
$$\lambda_r = s_r, \; |\phi_A^r\rangle = \sum_{a}U_{a,r}|a\rangle,\; |\phi_B^r\rangle = \sum_bV_{d,b}|b\rangle.$$

In [ ]:
import numpy as np

In [ ]:
# file: mps/state.py

DEFAULT_TOLERANCE = np.finfo(np.float64).eps


def vector2mps(ψ, dimensions, tolerance=DEFAULT_TOLERANCE):
    """Construct a list of tensors for an MPS that approximates the state ψ
    represented as a complex vector in a Hilbert space.

    Arguments:
    ----------
    ψ = wavefunction with \prod_i dimensions[i] elements
    dimension = list of dimensions of the Hilbert spaces that build ψ
    tolerance = truncation criterion for dropping Schmidt numbers"""

    def SchmidtSplit(ψ, tolerance):
        a, b = ψ.shape
        U, s, V = np.linalg.svd(ψ, full_matrices=False)
        s = _truncate_vector(s, tolerance)
        D = s.size
        return np.reshape(U[:, :D], (a, D)), np.reshape(s, (D, 1)) * V[:D, :]

    Da = 1
    dimensions = np.array(dimensions, dtype=np.int)
    Db = np.prod(dimensions)
    output = [0] * len(dimensions)
    for (i, d) in enumerate(dimensions):
        # We split a new subsystem and group the left bond dimension
        # and the physical index into a large index
        ψ = np.reshape(ψ, (Da * d, int(Db / d)))
        #
        # We then split the state using the Schmidt decomposition. This
        # produces a tensor for the site we are looking at and leaves
        # us with a (hopefully) smaller state for the rest
        A, ψ = SchmidtSplit(ψ, tolerance)
        output[i] = np.reshape(A, (Da, d, A.shape[1]))
        Da, Db = ψ.shape

    return output

Implicit in the canonicalization is a routine that truncates the singular values up to a tolerance, ensuring that the MPS we regenerate is not too large. We can use the norm-2 criterion for that. The idea is that the sum of the squares of the singular values is the norm of the full state
$$\|\psi\|^2 = \sum_r \lambda_r =: N$$
If we drop all values from $r_{cut}$ on, we make a norm-2 relative error
$$\varepsilon = \frac{1}{N} \sum_{r=r_{cut}} \lambda_r$$
We can study the relative error and use it to control our tolerance.

In [ ]:
# file: mps/state.py


def _truncate_vector(S, tolerance):
    #
    # Input:
    # - S: a vector containing singular values in descending order
    # - tolerance: truncation relative tolerance, which specifies an
    #   upper bound for the sum of the squares of the singular values
    #   eliminated. 0 <= tolerance <= 1
    #
    # Output:
    # - truncS: truncated version of S
    #
    if tolerance == 0:
        #log('--no truncation')
        return S
    # We sum all reduced density matrix eigenvalues, starting from
    # the smallest ones, to avoid rounding errors
    err = np.cumsum(np.flip(S, axis=0)**2)
    #
    # This is the sum of all values
    total = err[-1]
    #
    # we find the number of values we can drop within the relative
    # tolerance
    ndx = np.argmax(err >= tolerance*total)
    # and use that to estimate the size of the array
    # log('--S='+str(S))
    #log('--truncated to '+str(ndx))
    return S[0:(S.size - ndx)], (S[(S.size - ndx):]**2).sum()/total

We can apply this code to some significant states, such as the GHZ

In [ ]:
def test(state, dimensions):
    for (i,A) in enumerate(vector2mps(state, dimensions)):
        na, ni, nb = A.shape
        for a in range(na):
            for i in range(ni):
                for b in range(nb):
                    if np.abs(A[a,i,b]):
                        print("A[{},{},{}]={}".format(a,i,b,A[a,i,b]))

In [ ]:
test(np.array([1,0,0,1])/np.sqrt(2.0), [2,2])

In [ ]:
test(np.array([1,0,0,0,0,0,0,1])/np.sqrt(2.0), [2,2,2])

## Tensor arrays

The first step before creating more complicated structures is to ensure that we have the right logic when studying these tensor arrays. Our contract is as follows:

* The tensor network can be destructively modified. That is, we can replace tensors in the network after it has been created.

* The tensor network can be cloned, creating a fresh new copy that shares the same tensor.

* A clone of a tensor network only shares the tensors with its sibling. We can modify the sibling and the clone replacing tensors, without one affecting the other.

* We do not contemplate in-place modifications to the tensors themselves.

The class that implements this contract is shown below. It uses `__getitem__` and `__setitem__` to allow access to the class as if it was an array. It implements `__copy__()` to create fresh new copies, either with the library function `copy.copy()` or by directly using the method `copy()`.

In [ ]:
# file: mps/state.py


class TensorArray(object):
    """TensorArray class.

    This class provides the basis for all tensor networks. The class abstracts
    a one-dimensional array of tensors that is freshly copied whenever the
    object is cloned. Two TensorArray's can share the same tensors and be
    destructively modified.

    Attributes:
    size = number of tensors in the array
    """

    def __init__(self, data):
        """Create a new TensorArray from a list of tensors. 'data' is an
        iterable object, such as a list or other sequence. The list is cloned
        before storing it into this object, so as to avoid side effects when
        destructively modifying the array."""
        self._data = list(data)
        self.size = len(self._data)

    def __getitem__(self, k):
        #
        # Get MP matrix at position `k`. If 'A' is an MP, we can now
        # do A[k]
        #
        return self._data[k]

    def __setitem__(self, k, value):
        #
        # Replace matrix at position `k` with new tensor `value`. If 'A'
        # is an MP, we can now do A[k] = value
        #
        self._data[k] = value
        return value

    def __copy__(self):
        #
        # Return a copy of the MPS with a fresh new array.
        #
        return type(self)(self._data)

    def copy(self):
        """Return a fresh new TensorArray that shares the same tensor as its
        sibling, but which can be destructively modified without affecting it.
        """
        return self.__copy__()

## Simple MPS

An MPS is a tensor array that stores tensors with three indices. We will follow these conventions:

1. All tensors in MPS.data will have three indices, `A[α, i, β]`:
  - `(α, β)` are the virtual dimensions of the MPS
  - `i` is the physical dimension of the given site.
  
2. In general, we do not yet assume any normal form for the states.

3. We restrict ourselves to open-boundary-condition states, which have the following properties:
  - For the first site, `α=1`. For the last site `β=1`.
  - For the first site, α takes one value only, i.e. `α=0` in Python. For the last site `β=0` similarly.

All our implementations of MPS share the interface below, which provides methods for additional algorithms --scalar products, norms, expected values, etc-- that are developed in other components of the library.

In [ ]:
# file: mps/state.py


class MPS(TensorArray):
    """MPS (Matrix Product State) class.

    This implements a bare-bones Matrix Product State object with open
    boundary conditions. The tensors have three indices, A[α,i,β], where
    'α,β' are the internal labels and 'i' is the physical state of the given
    site.

    Attributes:
    size = number of tensors in the array
    """

    #
    # This class contains all the matrices and vectors that form
    # a Matrix-Product State.
    #
    def __init__(self, data):
        super(MPS, self).__init__(data)

    def dimension(self):
        """Return the total size of the Hilbert space in which this MPS lives."""
        return np.product([a.shape[1] for a in self._data])

    def tovector(self):
        """Return one-dimensional complex vector of dimension() elements, with
        the complete wavefunction that is encoded in the MPS."""
        return _mps2vector(self)

    @classmethod
    def fromvector(ψ, dimensions, **kwdargs):
        return MPS(vector2mps(ψ, dimensions, **kwdargs))

    def norm2(self):
        """Return the square of the norm-2 of this state, ‖ψ‖**2 = <ψ|ψ>."""
        return expectation.scprod(self, self)

    def expectation1(self, operator, n):
        """Return the expectation value of 'operator' acting on the 'n'-th
        site of the MPS."""
        return expectation.expectation1_non_canonical(self, operator, n)

    def expectation2(self, operator1, operator2, n):
        """Return the expectation value of 'operator1' and 'operator2' acting
        on the 'n'-th and 'n+1'-th sites of the MPS."""
        return expectation.expectation2_non_canonical(self, operator1,
                                                      operator2, n)

    def all_expectation1(self, operator):
        """Return all expectation values of 'operator' acting on all possible
        sites of the MPS."""
        return expectation.all_expectation1_non_canonical(self, operator)

### Convert an MPS into a vector

This is our first algorithm. We write a function that convers an MPS into a complex vector $\Psi$ with all the components in it. The algorithm implements the full contraction of the tensors, from left to right (i.e. from position 0 to position L-1, where L is the `mps.size`).

For instance, if the MPS has two sites of dimension 2, the MPS will have two tensors $A_{0i\alpha}$ and $B_{\alpha{j}0}$ that are contracted together to give the state

$$|\psi\rangle = \sum_{i,j,\alpha,\beta} A_{0i\alpha}B_{\alpha{j}0}|i\rangle\otimes|j\rangle.$$

In [ ]:
# file: mps/state.py


def _mps2vector(data):
    #
    # Input:
    #  - data: list of tensors for the MPS (unchecked)
    # Output:
    #  - Ψ: Vector of complex numbers with all the wavefunction amplitudes
    #
    # We keep Ψ[D,β], a tensor with all matrices contracted so far, where
    # 'D' is the dimension of the physical subsystems up to this point and
    # 'β' is the last uncontracted internal index.
    #
    Ψ = np.ones((1, 1,))
    D = 1
    for (i, A) in enumerate(data):
        α, d, β = A.shape
        Ψ = np.einsum('Da,akb->Dkb', Ψ, A)
        D = D * d
        Ψ = np.reshape(Ψ, (D, β))
    return Ψ.reshape((Ψ.size,))

Given our conventions, the wavefunction will be ordered as follows

$$\Psi = \left(\begin{array}{c}
\sum_\alpha A_{00\alpha} B_{\alpha00} \\
\sum_\alpha A_{00\alpha} B_{\alpha10} \\
\sum_\alpha A_{01\alpha} B_{\alpha00} \\
\sum_\alpha A_{01\alpha} B_{\alpha10}
\end{array}\right).$$

In [ ]:
_mps2vector([np.reshape([1, 2], (1, 2, 1)), np.reshape([3, 5], (1, 2, 1))])

This is the same convention as used by `np.kron()`

In [ ]:
np.kron([1, 2], [3, 5])

----

# Tests

Below we offer a minimal test framework for our matrix product state objects, that is part of our library's test suite. The tests can be run directly from here or from the test suite.

The first set of tests refers to the TensorArray object and verifies the contract we defined above.

In [ ]:
# file: mps/test/test_mps.py

import unittest


class TestTensorArray(unittest.TestCase):

    def setUp(self):
        self.product_state = [np.reshape([1.0, 2.0], (1, 2, 1)),
                              np.reshape([3.0, 5.0], (1, 2, 1)),
                              np.reshape([7.0, 11.0], (1, 2, 1))]

    def test_independence(self):
        #
        # If we create a TestArray, it can be destructively modified without
        # affecting it original list.
        #
        data = self.product_state.copy()
        A = TensorArray(data)
        for i in range(A.size):
            A[i] = np.reshape([13, 15], (1, 2, 1))
            self.assertTrue(np.all(A[i] != data[i]))
            self.assertTrue(np.all(data[i] == self.product_state[i]))

    def test_copy_independence(self):
        #
        # If we clone a TestArray, it can be destructively modified without
        # affecting its sibling.
        #
        A = TensorArray(self.product_state.copy())
        B = A.copy()
        for i in range(A.size):
            A[i] = np.reshape([13, 15], (1, 2, 1))
            self.assertTrue(np.all(A[i] != B[i]))
            self.assertTrue(np.all(B[i] == self.product_state[i]))

    def test_sharing(self):
        #
        # The clone of a TensorArray shares the same tensors
        #
        data = [x.copy() for x in self.product_state]
        A = TensorArray(data)
        B = A.copy()
        for i in range(A.size):
            A[i][0,0,0] = 17.0
            self.assertTrue(np.all(A[i] == B[i]))

In [ ]:
suite1 = unittest.TestLoader().loadTestsFromNames(['__main__.TestTensorArray'])
unittest.TextTestRunner(verbosity=2).run(suite1);